In [39]:
import os
import torch
from torch import nn
from peft import LoraConfig, TaskType, get_peft_model


class ModelForSourceCodeEmbedding(nn.Module):
    def __init__(self, model_name, normalize=True):
        super(ModelForSourceCodeEmbedding, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.normalize = normalize

    def forward(self, **kwargs):
        model_output = self.model(**kwargs)
        embeddings = self.average_pool(model_output, kwargs.get("attention_mask"))
        if self.normalize:
            embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
        return embeddings

    def average_pool(self, model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return (torch.sum(token_embeddings * input_mask_expanded, 1) /
                torch.clamp(input_mask_expanded.sum(1), min=1e-9))

    def __getattr__(self, name: str):
        try:
            return super().__getattr__(name)
        except AttributeError:
            return getattr(self.model, name)

In [40]:
from transformers import AutoModel, AutoTokenizer


model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = ModelForSourceCodeEmbedding(model_name)

In [41]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    bias="none",
    task_type=TaskType.FEATURE_EXTRACTION,
    target_modules=["key", "query", "value"],
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 442,368 || all params: 125,088,000 || trainable%: 0.3536


In [42]:
torch.cuda.empty_cache()

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device);

In [44]:
def get_cosing(q1_embs, q2_embs):
    return torch.sum(q1_embs * q2_embs, axis=1)

def get_loss(cosine_score, labels):
    return torch.mean(torch.square(labels * (1 - cosine_score) + torch.clamp((1 - labels) * cosine_score, min=0.0)))

def threshold(x):
    return 1 if x > 0.5 else 0

In [45]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm


train_cases = ["case-01", "case-02", "case-03", "case-04", "case-05"]
test_cases = ["case-06", "case-07"]

train_df = pd.DataFrame(data=None, columns=["original_code_file", "secondary_code_file", "label"])
train_df_index = 0

def add_to_train_df(original_code_file, secondary_code_file, label):
    global train_df_index
    train_df.loc[train_df_index] = [original_code_file, secondary_code_file, label]
    train_df_index += 1
    
def get_train_data(original_file_path, non_plagiarized_files_paths, plagiarized_files_paths):
    for non_plagiarized_file_path in non_plagiarized_files_paths:
        add_to_train_df(original_file_path, non_plagiarized_file_path, 0)
    for plagiarized_file_path in plagiarized_files_paths:
        add_to_train_df(original_file_path, plagiarized_file_path, 1)


# train-val cycle:
EPOCHS = 30

epochs_losses_train = []
epochs_losses_val = []
epochs_accuracies_train = []
epochs_accuracies_val = []

for epoch in range(EPOCHS):
    train_total_loss = 0
    train_correct = 0
    
    val_total_loss = 0
    val_correct = 0
    
    for case in train_cases:
        original_file_path = os.path.join("plagiarism_dataset", case, "original", os.listdir(f"plagiarism_dataset/{case}/original")[0])
        
        non_plagiarized_files_paths = [os.path.join("plagiarism_dataset", case, "non-plagiarized", file) for file in os.listdir(f"plagiarism_dataset/{case}/non-plagiarized")]
        non_plagiarized_files_paths = [os.path.join(file_path, os.listdir(file_path)[0]) for file_path in non_plagiarized_files_paths]
        
        plagiarized_files_paths = [os.path.join(dp, f) for dp, dn, filenames in os.walk(os.path.join("plagiarism_dataset", case, "plagiarized")) for f in filenames if os.path.splitext(f)[1] == '.java']
        
        get_train_data(original_file_path, non_plagiarized_files_paths, plagiarized_files_paths)
        train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
        X_train, X_test, y_train, y_test = train_test_split(train_df[["original_code_file", "secondary_code_file"]], train_df["label"], test_size=0.2, random_state=42)
    
        ros = RandomOverSampler(random_state=42)
        X_train, y_train = ros.fit_resample(X_train, y_train)
        
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)
    
        # train:
        optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
        loss = nn.CrossEntropyLoss()
        
        total_loss = 0
        correct = 0
        
        model.train()
        bar = tqdm(range(len(X_train)))
        for i in range(len(X_train)):
            original_code = open(X_train["original_code_file"][i], "r").read()
            secondary_code = open(X_train["secondary_code_file"][i], "r").read()
            
            q1 = tokenizer(original_code, return_tensors="pt", max_length=512, truncation=True)
            q2 = tokenizer(secondary_code, return_tensors="pt", max_length=512, truncation=True)
            
            q1 = {k: v.to(device) for k, v in q1.items()}
            q2 = {k: v.to(device) for k, v in q2.items()}
            
            label = torch.tensor([y_train[i]])
            label = label.to(device)
            
            optimizer.zero_grad()
            q1 = model(**q1)
            q2 = model(**q2)
            loss_value = get_loss(get_cosing(q1, q2).to(device), label)
            total_loss += loss_value.item()
            loss_value.backward()
            optimizer.step()
        
            prediction = threshold(get_cosing(q1, q2).item())
            if prediction == label:
                correct += 1
            bar.update(1)
        
        train_total_loss += total_loss / len(X_train)
        train_correct += correct / len(X_train)
    
        # print(f"Case: {case}, Loss train: {total_loss / len(X_train)}")
        # print(f"Case: {case}, Accuracy train: {correct / len(X_train)}")
        
        # val:
        total_loss = 0
        correct = 0
        
        model.eval()
        bar = tqdm(range(len(X_test)))
        for i in range(len(X_test)):
            # print(i)
            # print(X_test)
            original_code = open(X_test["original_code_file"][i], "r").read()
            secondary_code = open(X_test["secondary_code_file"][i], "r").read()
            
            q1 = tokenizer(original_code, return_tensors="pt", max_length=512, truncation=True)
            q2 = tokenizer(secondary_code, return_tensors="pt", max_length=512, truncation=True)
            
            q1 = {k: v.to(device) for k, v in q1.items()}
            q2 = {k: v.to(device) for k, v in q2.items()}
            
            label = torch.tensor([y_test[i]])
            label = label.to(device)
            
            q1 = model(**q1)
            q2 = model(**q2)
            loss_value = get_loss(get_cosing(q1, q2).to(device), label)
            total_loss += loss_value.item()
            
            prediction = threshold(get_cosing(q1, q2).item())
            if prediction == label:
                correct += 1
            bar.update(1)
        
        val_total_loss += total_loss / len(X_test)
        val_correct += correct / len(X_test)
        
        # print(f"Case: {case}, Loss val: {total_loss / len(X_test)}")
        # print(f"Case: {case}, Accuracy val: {correct / len(X_test)}")
    
    epochs_losses_train.append(train_total_loss / 5)
    print(f"Epoch: {epoch}, Loss train: {train_total_loss / 5}")
    epochs_accuracies_train.append(train_correct / 5)
    print(f"Epoch: {epoch}, Accuracy train: {train_correct / 5}")
    
    epochs_losses_val.append(val_total_loss / 5)
    print(f"Epoch: {epoch}, Loss val: {val_total_loss / 5}")
    epochs_accuracies_val.append(val_correct / 5)
    print(f"Epoch: {epoch}, Accuracy val: {val_correct / 5}")

100%|██████████| 62/62 [00:07<00:00,  8.12it/s]

100%|██████████| 150/150 [00:16<00:00,  8.82it/s]

100%|██████████| 234/234 [00:35<00:00,  6.63it/s]

100%|██████████| 324/324 [00:43<00:00,  7.43it/s]

100%|██████████| 404/404 [00:53<00:00,  7.49it/s]

100%|██████████| 66/66 [00:04<00:00, 11.84it/s]

Epoch: 0, Loss train: 0.24139840061505877
Epoch: 0, Accuracy train: 0.6883978073020978
Epoch: 0, Loss val: 0.5054344456601892
Epoch: 0, Accuracy val: 0.4973986013986014


100%|██████████| 462/462 [00:58<00:00,  7.92it/s]

100%|██████████| 556/556 [01:11<00:00,  7.82it/s]

100%|██████████| 644/644 [01:38<00:00,  6.52it/s]

100%|██████████| 726/726 [01:39<00:00,  7.31it/s]

100%|██████████| 816/816 [01:49<00:00,  7.43it/s]

 99%|█████████▉| 131/132 [00:09<00:00, 13.64it/s]

Epoch: 1, Loss train: 0.11417198436405154
Epoch: 1, Accuracy train: 0.8458112704647516
Epoch: 1, Loss val: 0.7935368486062767
Epoch: 1, Accuracy val: 0.3601776754319127


100%|██████████| 862/862 [01:52<00:00,  7.66it/s]

100%|██████████| 962/962 [02:03<00:00,  7.82it/s]

100%|██████████| 1048/1048 [02:23<00:00,  7.29it/s]

100%|██████████| 1130/1130 [02:32<00:00,  7.39it/s]

100%|██████████| 1194/1194 [02:40<00:00,  7.42it/s]

100%|██████████| 197/197 [00:12<00:00, 14.82it/s]

...

Epoch: 4, Loss train: 0.0026833836506269864
Epoch: 4, Accuracy train: 0.9969225156486875
Epoch: 4, Loss val: 0.015417607521910157
Epoch: 4, Accuracy val: 0.983318850654617


100%|██████████| 2090/2090 [04:32<00:00,  7.67it/s]

100%|██████████| 2186/2186 [04:48<00:00,  7.57it/s]

100%|██████████| 2250/2250 [05:00<00:00,  7.48it/s]

100%|██████████| 2328/2328 [05:08<00:00,  7.54it/s]

100%|██████████| 2426/2426 [05:23<00:00,  7.50it/s]

100%|██████████| 394/394 [00:26<00:00, 17.62it/s]

Epoch: 5, Loss train: 0.0010704681724491246
Epoch: 5, Accuracy train: 0.9988233173054798
Epoch: 5, Loss val: 0.0004401655004672751
Epoch: 5, Accuracy val: 1.0


100%|██████████| 2484/2484 [05:29<00:00,  7.55it/s]

100%|██████████| 2576/2576 [05:38<00:00,  7.61it/s]

100%|██████████| 2670/2670 [05:58<00:00,  7.45it/s]

100%|██████████| 2742/2742 [06:11<00:00,  7.39it/s]

 66%|██████▌   | 1870/2842 [04:12<02:15,  7.16it/s]

KeyboardInterrupt: 